# Data Processing

In [5]:
import numpy as np
import h5py
from scipy import ndimage
import os

## Load the data

In [6]:
os.getcwd()

'/Users/blaze/OneDrive/Work/HealthCare/Prostate MRI'

In [18]:
def _load():
    if not os.path.exists("./data/images.h5"):

        images = []
        for i in range(1, 2201):
            now_file_path = "./data/Image/IM" + str(i) + ".png"
            image = np.array(ndimage.imread(now_file_path, flatten=False))
            images.append(image)  # images shape=(m,64,64,3)
        images = np.array(images, copy=True)
        file = h5py.File('./data/images.h5', 'w')  # Create HDF5 database
        file.create_dataset('images', data=images)  # Write into database
        file.close()
    else:
        with h5py.File('./data/images.h5', 'r') as flie:
            images = flie.get("images")
            images = np.array(images, dtype=np.float32)

    if not os.path.exists("./data/labels.h5"):
        labels = []
        for i in range(1, 2201):
            now_file_path = "./data/Label/Label" + str(i) + ".png"
            label = np.array(ndimage.imread(now_file_path, flatten=False))
            labels.append(label)  # images shape=(m,64,64,3)
        labels = np.array(labels, copy=True)
        file = h5py.File('./data/labels.h5', 'w')  # Create HDF5 database
        file.create_dataset('labels', data=labels)  # Write into database
        file.close()

    else:
        with h5py.File('./data/labels.h5', 'r') as flie:
            labels = flie.get("labels")
            labels = np.array(labels, dtype=np.float32)

    images /= 255
    train_image = np.expand_dims(images, -1)
    print(train_image.shape)
    train_label = np.expand_dims(labels, -1)
    print(train_label.shape)

    return train_image, train_label



In [10]:
def get_data():
    image, label = _load()
    label[label == 128] = 1  # bladder wall area
    label[label == 255] = 2  # tumor area

    # one_hot processing: the whiter area is the tumor
    # label shape = (n,512,512,1)
    print(label.shape)
    label = encode_one_hot(label, 3)

    return image, label

In [11]:
def encode_one_hot(x, classes_num=3):
    """
    :param array x: ,Single channel picture.
    :param int classes_num: Dimension of one_hot or Classification number.
    :return: label of one_hot
    :rtype: narray
    """
    if x.shape[-1] == 1:
        x_tiled = np.tile(x, (1, 1, 1, classes_num))

        for i in range(classes_num):
            x_tiled[:, :, :, i] = np.where(x_tiled[:, :, :, i] == i, 1, 0)

    else:
        raise IndexError("The last dimension is not 1")
    return x_tiled

# CNN Learning

In [12]:
import tensorflow as tf
import numpy as np


class UNetKeras(object):

    def __init__(self, height=512, width=512, channel=1, classes=3):
        """
        U-net

        :param height int: The height of the picture
        :param width int: The width of the picture
        :param channel int: The number of channels in the picture,default is 1
        :param classes int: Classification number
        """
        inputs = tf.keras.layers.Input((height, width, channel))
        conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
        pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
        conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
        pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
        conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
        pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
        conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
        drop4 = tf.keras.layers.Dropout(0.5)(conv4)
        pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(
            pool4)
        conv5 = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(
            conv5)
        drop5 = tf.keras.layers.Dropout(0.5)(conv5)

        up6 = tf.keras.layers.Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            tf.keras.layers.UpSampling2D(size=(2, 2))(drop5))
        merge6 = tf.keras.layers.Concatenate(axis=3)([drop4, up6])
        conv6 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(
            merge6)
        conv6 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

        up7 = tf.keras.layers.Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            tf.keras.layers.UpSampling2D(size=(2, 2))(conv6))
        merge7 = tf.keras.layers.Concatenate(axis=3)([conv3, up7])
        conv7 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(
            merge7)
        conv7 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

        up8 = tf.keras.layers.Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            tf.keras.layers.UpSampling2D(size=(2, 2))(conv7))
        merge8 = tf.keras.layers.Concatenate(axis=3)([conv2, up8])
        conv8 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(
            merge8)
        conv8 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

        up9 = tf.keras.layers.Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            tf.keras.layers.UpSampling2D(size=(2, 2))(conv8))
        merge9 = tf.keras.layers.Concatenate(axis=3)([conv1, up9])
        conv9 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
        conv9 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv9 = tf.keras.layers.Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        # The last layer is the softmax layer. Classes (categorized classes) is the output path. 
        conv10 = tf.keras.layers.Conv2D(classes, 1, activation='softmax', padding='same',
                                        kernel_initializer='he_normal')(conv9)

        self.model = tf.keras.Model(inputs=inputs, outputs=conv10)

    def compile(self, optimizer=tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.90, beta_2=0.90),
                loss="categorical_crossentropy",
                metrics=['accuracy']):
        self.model.compile(optimizer=optimizer,
                           loss=loss,
                           metrics=metrics)

    def fit(self, x=None,
            y=None,
            batch_size=None,
            epochs=1,
            verbose=1,
            callbacks=None,
            validation_split=0.,
            validation_data=None,
            shuffle=True,
            class_weight=None,
            sample_weight=None,
            initial_epoch=0,
            steps_per_epoch=None,
            validation_steps=None, ):
        self.model.fit(x, y, batch_size, epochs, verbose, callbacks, validation_split, validation_data, shuffle,
                       class_weight, sample_weight, initial_epoch, steps_per_epoch, validation_steps)

    def predict(self, x, batch_size=32, verbose=1, steps=None):
        pred = self.model.predict(x, batch_size=batch_size, verbose=verbose, steps=steps)
        return np.argmax(pred, axis=-1)



# Run the Model

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf

In [20]:
print("=========          Get data            =========")
X, y = get_data()

=========          Get data            =========
(2200, 512, 512, 1)
(2200, 512, 512, 1)
(2200, 512, 512, 1)


In [30]:
X[0]

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)

In [25]:
print("=========Split train sets and test sets=========")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=2018)
X_train

=========Split train sets and test sets=========


array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [21]:
print("=========          Build model         =========")
model = UNetKeras()
model.compile()

=========          Build model         =========
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [33]:
print("=========       Start train model      =========")
ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint("./model/val_best_model.h5", monitor="val_loss", verbose=1,save_best_only=True)
model.fit(X_train, y_train, batch_size=1, epochs=10, validation_split=0.01, callbacks=[ModelCheckpoint])

=========       Start train model      =========
Train on 2156 samples, validate on 22 samples
Epoch 1/10
2156/2156 [==============================] - 43060s 20s/step - loss: 0.2028 - acc: 0.9874 - val_loss: 0.2242 - val_acc: 0.9861

Epoch 00001: val_loss improved from inf to 0.22420, saving model to ./model/val_best_model.h5
Epoch 2/10
2156/2156 [==============================] - 47080s 22s/step - loss: 0.2028 - acc: 0.9874 - val_loss: 0.2242 - val_acc: 0.9861

Epoch 00002: val_loss improved from 0.22420 to 0.22420, saving model to ./model/val_best_model.h5
Epoch 3/10
2156/2156 [==============================] - 44118s 20s/step - loss: 0.2028 - acc: 0.9874 - val_loss: 0.2242 - val_acc: 0.9861

Epoch 00003: val_loss improved from 0.22420 to 0.22420, saving model to ./model/val_best_model.h5
Epoch 4/10
2156/2156 [==============================] - 44950s 21s/step - loss: 0.2028 - acc: 0.9874 - val_loss: 0.2242 - val_acc: 0.9861

Epoch 00004: val_loss improved from 0.22420 to 0.22420, sav

KeyboardInterrupt: 

In [32]:
os.getcwd()

'/Users/blaze/OneDrive/Work/HealthCare/Prostate MRI'